In [1]:
from IPython.core.display import display, HTML, Markdown
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Select Webpages To Create

There are 4 options for webpages to create with this script:

1. Ratings (a webpage of the ratings that you have recieved, including links to those reviews)
2. Reviews (a webpage of the reviews that you have submitted, including links to those reviews)
3. Your Submissions (a webpage of the projects that you have submitted as a student, including links to the reviews that you recieved)
4. Combined Ratings and Reviews (1. and 2. merged)

By creating a list, called `create`, you select which to create.

In [3]:
# For example: create = [1,2,3,4] will create all 4 types of webpages

# create = [1,2,3,4]
create = [1,2,4]

# Select Data For Webpages

By default, the _Ratings_ and _Reviews_ JSONs are limited to the last 24 entries.

You can over-ride those defaults by selecting a _start date_ for the _Ratings_ JSON and a _limit_ (the number of reviews to download) for the _Reviews_ JSON. If you want to keep the default (which is the quickest option) use an empty string for the variables below (if you have downloaded all of your information once, then just use the default options).

In [4]:
## Default
start_date = ''
limit = ''

## Optional
# Uncomment and change '2018-07-14' and '300' to the values that you want:
# start_date = '?start_date=2018-07-14'
# limit = '?limit=320'

# Setup Information

[Download chromedriver](http://chromedriver.chromium.org/downloads) - required

In [5]:
import datetime
time_date = datetime.datetime.now().strftime("%I_%M%p_on_%b_%d")

#============> CHANGE THIS
chromedriver_local_path = "/path/to/chromedriver"
logon_webpage = 'https://auth.udacity.com/sign-in?next=https%3A%2F%2Fmentor-dashboard.udacity.com%2Freviews%2Foverview'

#============> CHANGE THIS
email = "email@email.com"
#============> CHANGE THIS
password = "password"


import datetime
time_date = datetime.datetime.now().strftime("%I_%M%p_on_%b_%d")

## Feedbacks page Information
#============> OPTIONAL USE SPECIFIC LIMIT - see above - (default is only 24)
feedbacks_webpage = 'https://review-api.udacity.com/api/v1/me/student_feedbacks.json{}'.format(start_date)

# variables relevant to this page:
f_date = 'updated_at';f_file="ratings_{}.csv";f_html="Ratings_{}.html"
if not start_date:
    f_html="Ratings_{}.html".format(time_date)

## Review page Information
#============> OPTIONAL USE SPECIFIC LIMIT - see above - (default is only 24)
review_webpage = 'https://review-api.udacity.com/api/v1/me/submissions/completed.json{}'.format(limit)
# variables relevant to this page:
r_date='completed_at'; r_file="reviews_{}.csv";r_html="Reviews_{}.html"
if not start_date:
    r_html="Reviews_{}.html".format(time_date)

## Convert Markdown to HTML? 
#============> The python 'markdown' module has to be installed.
convertMarkdown = True

## individual submissions page Information
submissions_webpage = 'https://review-api.udacity.com/api/v1/me/submissions'
# variables relevant to this page:
s_date='completed_at'; s_file="my_submissions_{}.csv";s_html="my_submissions_{}.html"
if not start_date:
    s_html="My_Submissions_{}.html".format(time_date)

## Combined Information
# variables relevant to this page:
c_html = 'Combined_Reviews_{}.html'
if not start_date:
    c_html="Combined_Reviews_{}.html".format(time_date)

## Download JSONs

In [6]:
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import os
import datetime
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import json


__author__ = 'Myles'

class MentorReviewStudentFeedbacks(object):
    '''setUp() = Access Udacity's Mentor Review Page
       downloadFeedbacks() = Download feedback information
       downloadReviews() = Download review information
    '''
       

    def __init__(self,chromedriver_local_path,logon_webpage,email,password):
        # Allow Notifications
        self.opt = Options()
        # Pass the argument 1 to allow and 2 to block
        self.opt.add_experimental_option("prefs", { 
        #     "profile.default_content_setting_values.media_stream_mic": 1, 
        #     "profile.default_content_setting_values.media_stream_camera": 1,
        #     "profile.default_content_setting_values.geolocation": 1, 
            "profile.default_content_setting_values.notifications": 1 
        })
        # Setup
        self.chromedriver_local_path = chromedriver_local_path
        self.logon_webpage = logon_webpage
        self.email = email
        self.password = password
        self.driver = webdriver.Chrome(self.chromedriver_local_path,chrome_options=self.opt)
        self.driver.set_window_position(0, 0)
        self.driver.set_window_size(1466, 1124)

    def setUp(self):
        # quick sign-in
        self.driver.get(self.logon_webpage)
        time.sleep(1)
        self.email_elem = WebDriverWait(self.driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='email']")))
        self.email_elem.send_keys(self.email)
        self.password_elem = WebDriverWait(self.driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='password']")))
        self.password_elem.send_keys(self.password + Keys.ENTER)
        time.sleep(2)
        
    def downloadpandasDF(self,webpage,date,filename,download=True):
        time_date = datetime.datetime.now().strftime("%H_%M%_on_%b_%d")
        # access review content page
        self.driver.get(webpage)
        content = self.driver.find_element_by_xpath("html").text
        feedback_json = json.loads(content)
        feedback = pd.DataFrame(feedback_json)
        feedback = pd.concat([feedback.drop(['project'], axis=1), feedback['project'].apply(pd.Series).add_suffix('_project')], axis=1)
        feedback[date] = pd.to_datetime(feedback[date])
        feedback["Date"] = feedback[date].map(lambda x: '{}'.format(x.strftime("%m/%d/%y at %I:%M%p")))
        feedback.sort_values(by=date, inplace=True, ascending=False)
        if download:
            feedback.to_csv(filename.format(time_date), encoding='utf-8', index=False)
        return feedback
    
    def tearDown(self):
        self.driver.quit()
        

In [7]:
# create instance
discourse = MentorReviewStudentFeedbacks(chromedriver_local_path,logon_webpage,email,password)

# log on to dashboard
discourse.setUp()
reviews_df = discourse.downloadpandasDF(review_webpage,r_date,r_file)
feedback_df = discourse.downloadpandasDF(feedbacks_webpage,f_date,f_file)
my_submissions_df = discourse.downloadpandasDF(submissions_webpage,s_date,s_file)
# Quit Driver
discourse.tearDown()

## Create Webpage

In [9]:
def find_nth(haystack, needle, n):
    '''find nth occurance of a substring in a string'''
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

In [10]:
class CreateReviewsWebpage(object):
    '''setUp() = Access Udacity's Mentor Review Page
       downloadFeedbacks() = Download feedback information
       downloadReviews() = Download review information
    '''
    def __init__(self,reviewList,fileName,title = "Feedback Page", collapse =[]):
        '''Required inputs: A list of lists, a filename, and a page name'''
        self.reviewList = reviewList
        self.fileName = fileName
        self.title = title
        self.collapse = collapse
        
    def createTable(self, listOfLists):
        """Create the HTML table, from the list of lists generated from the pandas dataframe (or other methods)"""
        htmlTable = "<table id='pyTable'><tr>"
        for ind, rows in enumerate(listOfLists):
            for col, data in enumerate(rows):
                # add either the table headers (list 0) or the table rows (all other list items)
                if ind == 0:
                    htmlTable += "<th>{}</th>".format(data)
                else:
                    if col in self.collapse:
                        if find_nth(data, '</h1>', 1) > 0:
                            strIndex = find_nth(data, '</h1>', 1) + 6
                        elif len(data) > 200:
                            strIndex = len(data.split('. ',2)[0]) + 2
                        else:
                            strIndex = len(data)
                        htmlTable += "<td><button class='collapsible'>{}</button><div class='content'>{}</div></td>".format(data[:strIndex],data[strIndex:])
                    else:
                        htmlTable += "<td>{}</td>".format(data)
            
            htmlTable += "</tr><tr>".format(data)
        # remove the last <tr>, close the table
        htmlTable = htmlTable[:-4] + "</table>"
        return htmlTable
    
    def createContents(self):
        '''Create the HTML page. The text will be the HTML table generated from createTable and the pandas data'''
        createdTable = self.createTable(self.reviewList)
        contents = """<!DOCTYPE html PUBLIC '-//W3C//DTD HTML 4.01 Transitional//EN'>
            <html>
            <head>
              <meta content='text/html; charset=ISO-8859-1'
             http-equiv='content-type'>
              <title>{0}</title>
            </head>
            <style>
            body {{
            width:100vw !important;
            }}
            table {{
            width:100vw !important;
            }}
            th {{
            width: 5vw !important;
            }}
            
            .collapsible {{
              background-color: #777;
              color: white;
              cursor: pointer;
              padding: 18px;
              width: {1}vw !important;
              border: none;
              text-align: left;
              outline: none;
              font-size: 15px;
            }}

            .active, .collapsible:hover {{
              background-color: #555;
            }}

            .collapsible:after {{
              content: '\\002B';
              color: white;
              font-weight: bold;
              float: right;
              margin-left: 5px;
            }}

            .active:after {{
              content: '\\2212';
            }}

            .content {{
              padding: 0 18px;
              max-height: 0;
              width: {1}vw !important;
              overflow: hidden;
              transition: max-height 0.2s ease-out;
              background-color: #f1f1f1;
            }}
            #pyTable {{
                font-family: 'Trebuchet MS', Arial, Helvetica, sans-serif;
                border-collapse: collapse;
                width: 100%;
            }}
            #pyTable td, #pyTable th {{
                border: 1px solid #ddd;
                padding: 8px;
            }}
            #pyTable tr:nth-child(even){{background-color: #f2f2f2;}}
            #pyTable tr:hover {{background-color: #ddd;}}
            #pyTable th {{
                padding-top: 12px;
                padding-bottom: 12px;
                text-align: left;
                background-color: #4CAF50;
                color: white;
            }}
            </style>
            <body>{2}
            <script>
            var coll = document.getElementsByClassName("collapsible");
            var i;

            for (i = 0; i < coll.length; i++) {{
              coll[i].addEventListener("click", function() {{
                this.classList.toggle("active");
                var content = this.nextElementSibling;
                if (content.style.maxHeight){{
                  content.style.maxHeight = null;
                }} else {{
                  content.style.maxHeight = content.scrollHeight + "px";
                }} 
              }});
            }}
            </script>
            </body>
            </html>""".format(self.title, ((100-(5*len(self.reviewList[0])))/len(self.collapse))-1, createdTable)
        return contents

    def strToFile(self):
        """Create the HTML page. Then write the HTML file with the given name and the given contents."""
        self.contents = self.createContents()
        output = open(self.fileName,"w")
        output.write(self.contents)
        output.close()

    def browseLocal(self):
        '''Open chrome webbrowser with the local file created above
        with the given filename.'''
        import webbrowser, os.path
        self.strToFile()
        # Note: remove .get(using='chrome') if you are using your default browser to log on to udacity (and it is not chrome)
        webbrowser.get(using='chrome').open("file:///" + os.path.abspath(self.fileName),new=2) #elaborated for Mac
        


# Create Ratings Webpage(s)

## Webpage columns to include:

For **ratings** the column options are: 
    
       ['body', 'created_at', 'id', 'rating', 'read_at', 'submission_id',
       'updated_at', 'id_project', 'name_project', 'visible_project',
       'is_cert_project_project', 'is_career_project', 'Date', 'link']


In [11]:
includeCols = ['Date','rating','name_project','Link','body']
# Optional Headers (otherwise column names are used). Has to have same len as includeCols
optHeaders = ['Date','Rating','Project','Link','Comment']

# choose columns to sort by
sortBy = ['updated_at']
# choose ordering. Has to have same len as sortBy
sortOrder = [False]

# Markdown Columns
markdownCols = ['body']

# collapsible cells
collapse = [4]

# choose HTML name
fileName = f_html
linkID = 'submission_id'
pageTitle = 'Feedback Page'

### Function to set options and call CreateReviewsWebpage

In [12]:
#  All arguments set as defaults (change values above)
import markdown


def callCreateReviewsWebpage(df, byProject = True, linkID = linkID, pageTitle = pageTitle, sortBy=sortBy, sortOrder = sortOrder ,
                             includeCols = includeCols, defaultHeaders = False, optHeaders = optHeaders, fileName=fileName,
                            convertMarkdown = convertMarkdown, markdownCols = markdownCols, collapse=collapse):
    
    ## create instance of markdown
    md = markdown.Markdown()

    ## Sort by project name, then date (change to the option that you want):
    df.sort_values(by=sortBy, inplace=True, ascending=sortOrder)

    ## Create link (from submission number)
    try:
        df['Link'] = df[linkID].astype(int).astype(str).apply(lambda x: f"<a href=https://review.udacity.com/#!/reviews/{x} target='_blank'>{x}</a>")
    except:
        # if duplicate column names, select first
        df['Link'] = df[linkID].astype(int).astype(str).iloc[:,0].apply(lambda x: f"<a href=https://review.udacity.com/#!/reviews/{x} target='_blank'>{x}</a>")
    
    ## Convert Markdown to HTML
    if convertMarkdown:
        for cols in markdownCols:
            df[cols] = df[cols].apply(lambda x: md.convert(x) if isinstance(x,str) and x else "No Entry")
        

    ## Columns to include in HTML table
    dfHTML = df[includeCols].copy()
    
    if byProject:
        ## Create separate pages for each project
        for ind, grp in dfHTML.groupby('name_project'):

            ## create a list of lists of entries
            myList = grp.values.tolist()    

            ## add the column headers (will become table headers)
            if defaultHeaders:
                myList.insert(0,list(grp.columns))
            else:
                ## or add your own headers
                myList.insert(0,optHeaders)


            reviewWebpage = CreateReviewsWebpage(myList,fileName=fileName.format(ind),title=pageTitle, collapse=collapse)
            reviewWebpage.browseLocal()
    else:
        ## create a list of lists of entries
        myList = dfHTML.values.tolist()    

        ## add the column headers (will become table headers)
        if defaultHeaders:
            myList.insert(0,list(dfHTML.columns))
        else:
            ## or add your own headers
            myList.insert(0,optHeaders)


        reviewWebpage = CreateReviewsWebpage(myList,fileName=fileName, title=pageTitle, collapse=collapse)
        reviewWebpage.browseLocal()        

### Create Ratings Webpage

In [13]:
if 1 in create:
    callCreateReviewsWebpage(feedback_df)

# Create Review Webpage(s)

## Webpage columns to include:

For **reviews** the column options are: 

       ['annotation_urls', 'archive_url', 'assigned_at', 'checkmate_metadata',
       'commit_sha', 'completed_at', 'created_at', 'enrollment_id',
       'enrollment_node_key', 'escalated_at', 'files', 'general_comment',
       'grader_id', 'held_at', 'hidden', 'id', 'is_training', 'language',
       'nomination', 'notes', 'plagiarism_source_url',
       'previous_submission_id', 'price', 'project_id', 'repo_url', 'result',
       'result_reason', 'rubric_id', 'status', 'status_reason', 'training_id',
       'type', 'ungradeable_tag', 'updated_at', 'url', 'user_id', 'id_project',
       'name_project', 'visible_project', 'is_cert_project_project',
       'is_career_project', 'Date']

In [14]:
if 2 in create:
    includeCols = ['Date','name_project','Link','general_comment']
    # Optional Headers (otherwise column names are used). Has to have same len as includeCols
    optHeaders = ['Date','Project','Link','Comment']

    # choose columns to sort by
    sortBy = ['completed_at']
    # choose ordering. Has to have same len as sortBy
    sortOrder = [False]

    # Markdown Columns
    markdownCols = ['general_comment']

    # collapsible cells
    collapse = [3]

    # choose HTML name
    fileName = r_html
    linkID = 'id'
    pageTitle = 'Reviews Page'


    callCreateReviewsWebpage(reviews_df, linkID=linkID, pageTitle=pageTitle, sortBy=sortBy, sortOrder = sortOrder , includeCols = includeCols,
                             defaultHeaders = False, optHeaders = optHeaders, fileName=fileName, markdownCols = markdownCols, collapse=collapse)

# Create Personal Submissions Webpage

## Webpage columns to include:

For your **submissions** the column options are: 
    
       ['annotation_urls', 'archive_url', 'assigned_at', 'checkmate_metadata',
       'commit_sha', 'completed_at', 'created_at', 'enrollment_id',
       'enrollment_node_key', 'escalated_at', 'files', 'general_comment',
       'grader_id', 'held_at', 'hidden', 'id', 'is_training', 'language',
       'nomination', 'notes', 'plagiarism_source_url',
       'previous_submission_id', 'price', 'project_id', 'repo_url', 'result',
       'result_reason', 'rubric_id', 'status', 'status_reason', 'training_id',
       'type', 'ungradeable_tag', 'updated_at', 'url', 'user_id', 'id_project',
       'name_project', 'visible_project', 'is_cert_project_project',
       'is_career_project', 'Date', 'link']





In [15]:
if 3 in create:
    includeCols = ['Date','name_project','Link','escalated_at','general_comment']
    # Optional Headers (otherwise column names are used). Has to have same len as includeCols
    optHeaders = ['Date','Project','Link', 'Date Escalated','Comment']

    # choose columns to sort by
    sortBy = [ 'completed_at', 'name_project']
    # choose ordering. Has to have same len as sortBy
    sortOrder = [False, True]

    # Markdown Columns
    markdownCols = ['general_comment']

    # collapsible cells
    collapse = [4]

    # choose HTML name
    fileName = s_html
    linkID = 'id'
    pageTitle = 'My Submissions'
    callCreateReviewsWebpage(my_submissions_df,byProject=False,linkID=linkID, pageTitle=pageTitle, sortBy=sortBy, sortOrder = sortOrder, includeCols = includeCols,
                             defaultHeaders = False, optHeaders = optHeaders, fileName=fileName, markdownCols = markdownCols, collapse=collapse)

In [16]:
print('Number of:\nReviews {}\nRatings {}\nMy Submissions {}'.format(len(reviews_df),len(feedback_df),len(my_submissions_df)))

Number of:
Reviews 312
Ratings 190
My Submissions 40


# Create Combined Reviews and Ratings Webpage(s)

## Webpage columns to include, by merging columns from:

For **reviews** the column options are: 

       ['annotation_urls', 'archive_url', 'assigned_at', 'checkmate_metadata',
       'commit_sha', 'completed_at', 'created_at', 'enrollment_id',
       'enrollment_node_key', 'escalated_at', 'files', 'general_comment',
       'grader_id', 'held_at', 'hidden', 'id', 'is_training', 'language',
       'nomination', 'notes', 'plagiarism_source_url',
       'previous_submission_id', 'price', 'project_id', 'repo_url', 'result',
       'result_reason', 'rubric_id', 'status', 'status_reason', 'training_id',
       'type', 'ungradeable_tag', 'updated_at', 'url', 'user_id', 'id_project',
       'name_project', 'visible_project', 'is_cert_project_project',
       'is_career_project', 'Date']



For **ratings** the column options are: 
    
       ['body', 'created_at', 'id', 'rating', 'read_at', 'submission_id',
       'updated_at', 'id_project', 'name_project', 'visible_project',
       'is_cert_project_project', 'is_career_project', 'Date', 'link']


### To account for Can't Review entries - use "result_reason" (the explanation of why the review could not be reviewed)

In [17]:
if 4 in create:
    import numpy as np
    reviews_df['review_comment'] = np.where(reviews_df['general_comment'].str.contains("No Entry"),
                                             "<h1>Cannot Review</h1>  " + reviews_df['result_reason'],
                                            reviews_df['general_comment'] )

### Merge the columns that you want from the Reviews and Ratings dataframes (on the submission id)

In [18]:
if 4 in create:

    combined = pd.merge(reviews_df[['Date','name_project','review_comment','id', 'completed_at']],
                        feedback_df[['Date','rating','name_project','body','submission_id','created_at']],
                        how = 'left',left_on = 'id', right_on = 'submission_id')
    combined.rating.fillna("No\nRating", inplace=True)
    combined.Date_y.fillna("NA", inplace=True)

### Set up the webpage (in the same way as earlier)

In [19]:
if 4 in create:

    combined.columns = ['Review Date', 'name_project', 'Review Comment','Review ID', 'completed_at',
                        'Rating Date', 'Rating', 'Project Name', 'Rating Comment', 'ID Sub','created_at']

    includeCols = ['Review Date', 'Rating Date','Link', 'Rating', 'Rating Comment', 'name_project', 'Review Comment']
    # Optional Headers (otherwise column names are used). Has to have same len as includeCols
    optHeaders = ['Review Date', 'Rating Date','Link', 'Rating', 'Rating Comment', 'Project', 'Review Comment']

    # choose columns to sort by
    sortBy = [ 'completed_at', 'name_project']
    # choose ordering. Has to have same len as sortBy
    sortOrder = [False, True]

    # Markdown Columns
    markdownCols = ['Review Comment', 'Rating Comment']

    # collapsible cells
    collapse = [4,6]

    # choose HTML name
    fileName = c_html
    linkID = 'Review ID'
    pageTitle = 'Combined Reviews'
    callCreateReviewsWebpage(combined,byProject=True,linkID=linkID, pageTitle=pageTitle, sortBy=sortBy, sortOrder = sortOrder,includeCols = includeCols,
                             defaultHeaders = False, optHeaders = optHeaders, fileName=fileName, markdownCols = markdownCols, collapse=collapse)